## Long-term trends in Snow Cover in Ladakh, India

Snow cover controls the global and regional climate and releases fresh water in the hydrological cycle. Snow exhibits a strong negative correlation with atmospheric temperature. Changes in snow covered areas (SCA) alter the land-surface heating and the overlying atmosphere. Therefore, trends in SCA can be a crucial indicator of how the area is being changed.

‘The Himalaya,’ which describes India’s north extent, is the largest source of snow and ice cover outside the polar regions in the world. Several studies have indicated that the Himalayan region is warming faster than the global average over the last century. Other studies suggest that continuous increase in annual LST trends would contribute to accelerated retreating of glaciers and cause heatwave and drought conditions in lower elevations. 

## Study Area

Ladakh is a region administered by India as a union territory. It extends from the Siachen Glacier in the Karakoram range to the north to the main Great Himalayas to the south. It is the highest plateau in India with most of it being over 3,000 m. Ladakh is a high-altitude desert as the Himalayas create a rain shadow, generally denying entry to monsoon clouds. The main source of water is the winter snowfall on the mountains. 

<img src="./images/ladakh.png" alt="Study Area, Ladakh" style="width: 600px; height:450px"/>


## Objectives

1. Assess snow covered area (SCA) trends in the region of Ladakh between 2010 and 2022 for a period of 13 years. 
2. Classify the study area into different elevation classes to differentiate the impact of elevation on trends in SCA. 


## Methodology

The data needed for this analysis is obtained from the Planetary Computer Data Catalog. MODIS  Snow Cover Daily dataset is used to calculate SCA and NASADEM dataset is used for elevation.
From SC data, "NDSI_Snow_Cover" band is obtained which represents SC in percentage (0-100). From DEM data, "elevation" band is obtained and resampled to a resolution of 500 metres to match with MODIS data.

Snow cover data is obtained as an Xarray dataset using "odc.stac.load" method for the summer months (May, June, July, and August). The data is then grouped together monthly by calculation the mean snow cover percentage along the time dimension. It is then converted into binary SC with a threshold of 50%.

Elevation data is is obtained as an Xarray dataset using "stackstac.stac" method. Since there are multiple tiles, "stackstac.mosaic" method is used to mosaic the dataset. The data is then classified into 5 classes for elevation ranges 0-3000m (1), 3000-4000m (2), 4000-5000m (3), 5000m-6000m (4), 6000-9000m (5) respectively. To do this, "xarray.apply_ufunc" methods is used.

<img src="./images/ladakh_dem_classified.png" alt="Classified based on elevation, Ladakh" style="width: 700px; height:500px"/>


Only zones 2, 3 and 4 were used to compute zonal statistics as zones 1 and 5 constitute less than 15% of the study area. The method "xrspatial.zonal_stats" method is used to compute the number of SCA pixels in elevation zones which returns the statistics as a pandas dataframe. The results for each year are combined together and presented as a bar plot to visualize the trends.

## Results

### Zone 2 3000-4000m

![Snow cover trends in Zone 2](./images/class2_trend.png "Snow cover trends in Zone 2")

### Zone 3 4000m-5000m

![Snow cover trends in Zone 3](./images/class3_trend.png "Snow cover trends in Zone 3")

### Zone 4 5000m-6000m

![Snow cover trends in Zone 4](./images/class4_trend.png "Snow cover trends in Zone 4")

## Discussion

Zonal statistics confirm that the Himalayan region is experiencing glacial retreat, particularly at lower elevations, due to global warming. This trend is evident in zone 2, where a significant decreasing trend in snow cover is noted during June and July. Additionally, zone 3 show decreasing trend in July and August. While zone 4 maintains consistent snow cover through May, June, and July, but also reveals a decline in August. These patterns are consistent with rising temperatures in the region, especially during the months of July and August, as reported in scientific literature.

### Challenges Encountered

While performing zonal statistics, there were several challenges, primarily due to errors arising from data set comparisons. The process requires that both the zones raster and the values raster not only share the same projection and spatial bounds but also the data type. In my case, zones raster, which was a result of "xarray.apply_ufunc" method was not dask-based and had data type as "int64". In practice, these rasters are both of type "xarray.DataArray", yet the data within differs—one being a NumPy n-dimensional array, and the other a Dask array. To resolve this, it was necessary to convert the zones raster into a Dask array format, change the data type to "float64" and align spatially using "reproject_match" and then run "xrspatial.zonal_stats". 

The initial plan was to employ Landsat data collection for analysis, but the complexity of managing numerous images from various Landsat satellites, such as 8, 9, and 7, for the same area, led to an overwhelming search inventory. This complexity made computations, particularly calculating the mean over time dimension, prohibitively time-consuming and costly, despite employing distributed computing with lazy evaluation. Consequently, there's a compromise between the high resolution of images and the size of the study area. In this case with large study region, using Landsat data proved impractical. Thus, researchers must choose between high-resolution data for small areas, ensuring precision, or broader analyses with lower-resolution data that offer a more general overview.


### Suggested Improvements

The analysis, conducted over 13 years, could yield more pronounced trends if extended over a longer timeframe. Integrating additional datasets, such as land surface temperature and precipitation, would enhance our understanding of the relationships between these variables and their collective impact. Furthermore, utilizing higher resolution data could refine the analysis, though it may increase computational demands.